# MT5 Trading Bot - Analysis & Strategy Testing

This notebook provides comprehensive analysis tools for your MT5 trading bot including:
- Strategy backtesting
- Performance visualization
- Parameter optimization
- Risk analysis
- Live trading monitoring

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings('ignore')

# Import your trading bot modules
from bot import *
from strategy import *
from backtest import *
from utils import *

# Setup plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All modules imported successfully!")

## 2. MT5 Connection and Account Info

In [ ]:
# Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()

# MT5 credentials
MT5_ACCOUNT = os.getenv("MT5_ACCOUNT")
MT5_PASSWORD = os.getenv("MT5_PASSWORD")
MT5_SERVER = os.getenv("MT5_SERVER")

print(f"Account: {MT5_ACCOUNT}")
print(f"Server: {MT5_SERVER}")

# Connect to MT5
if connect_mt5(MT5_ACCOUNT, MT5_PASSWORD, MT5_SERVER):
    account_info = get_account_info()
    if account_info:
        print("\n📊 Account Information:")
        print(f"Balance: ${account_info['balance']:.2f}")
        print(f"Equity: ${account_info['equity']:.2f}")
        print(f"Leverage: 1:{account_info['leverage']}")
        print(f"Currency: {account_info['currency']}")
    else:
        print("❌ Failed to get account info")
else:
    print("⚠️ Running in simulation mode (MT5 not connected)")

## 3. Market Data Analysis

In [ ]:
# Trading symbols
SYMBOLS = ['EURUSD', 'GBPUSD', 'USDJPY']

def get_market_data(symbol, timeframe=mt5.TIMEFRAME_H1, bars=1000):
    """Get historical market data"""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, bars)
        if rates is not None:
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df.set_index('time', inplace=True)
            return df
    except Exception as e:
        print(f"Error getting data for {symbol}: {e}")
    return None

# Get data for all symbols
market_data = {}
for symbol in SYMBOLS:
    data = get_market_data(symbol)
    if data is not None:
        market_data[symbol] = data
        print(f"✅ {symbol}: {len(data)} bars loaded")
    else:
        print(f"❌ Failed to load {symbol}")

if not market_data:
    print("\n⚠️ No market data available. Creating sample data for demonstration...")
    # Create sample data for demo
    dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='H')
    for symbol in SYMBOLS:
        np.random.seed(42)  # For reproducible results
        prices = np.cumsum(np.random.randn(len(dates)) * 0.001) + 1.1000
        market_data[symbol] = pd.DataFrame({
            'open': prices,
            'high': prices + np.random.rand(len(dates)) * 0.002,
            'low': prices - np.random.rand(len(dates)) * 0.002,
            'close': prices + np.random.randn(len(dates)) * 0.001,
            'tick_volume': np.random.randint(100, 1000, len(dates))
        }, index=dates)
    print("📊 Sample data created for demonstration")

## 4. Price Visualization

In [ ]:
# Plot price charts for all symbols
fig, axes = plt.subplots(len(SYMBOLS), 1, figsize=(15, 5*len(SYMBOLS)))
if len(SYMBOLS) == 1:
    axes = [axes]

for i, symbol in enumerate(SYMBOLS):
    if symbol in market_data:
        df = market_data[symbol].tail(500)  # Last 500 bars
        axes[i].plot(df.index, df['close'], label='Close Price', linewidth=1)
        axes[i].set_title(f'{symbol} - Close Price (Last 500 bars)')
        axes[i].set_ylabel('Price')
        axes[i].grid(True, alpha=0.3)
        axes[i].legend()

plt.tight_layout()
plt.show()

# Print basic statistics
print("\n📈 Market Data Summary:")
for symbol in SYMBOLS:
    if symbol in market_data:
        df = market_data[symbol]
        print(f"\n{symbol}:")
        print(f"  Current Price: {df['close'].iloc[-1]:.5f}")
        print(f"  Daily Change: {((df['close'].iloc[-1] / df['close'].iloc[-25]) - 1) * 100:.2f}%")
        print(f"  Volatility (std): {df['close'].pct_change().std() * 100:.3f}%")

## 5. Strategy Testing

In [ ]:
# Test trading strategy on historical data
def test_strategy_on_data(symbol, df, params=None):
    """Test strategy and return signals"""
    if params is None:
        params = {
            'ema_period': 50,
            'rsi_period': 14,
            'rsi_overbought': 70,
            'rsi_oversold': 30,
            'macd_slow': 26,
            'macd_fast': 12,
            'macd_signal': 9
        }
    
    try:
        # Generate signals using your strategy
        signal = generate_trade_signal(df, symbol, params)
        return signal
    except Exception as e:
        print(f"Error testing strategy on {symbol}: {e}")
        return None

# Test strategy on all symbols
print("🧪 Testing Strategy on Historical Data...\n")
signals = {}
for symbol in SYMBOLS:
    if symbol in market_data:
        df = market_data[symbol]
        signal = test_strategy_on_data(symbol, df)
        signals[symbol] = signal
        
        if signal:
            print(f"📊 {symbol} Signal Analysis:")
            print(f"  Signal: {signal.get('signal', 'NONE')}")
            print(f"  Confidence: {signal.get('confidence', 0):.2f}")
            print(f"  Current Price: {signal.get('price', 0):.5f}")
            if signal.get('signal') in ['BUY', 'SELL']:
                print(f"  Entry Price: {signal.get('entry_price', 0):.5f}")
                print(f"  Stop Loss: {signal.get('stop_loss', 0):.5f}")
                print(f"  Take Profit: {signal.get('take_profit', 0):.5f}")
            print()
        else:
            print(f"❌ No signal generated for {symbol}\n")

## 6. Backtesting

In [ ]:
# Run comprehensive backtests
print("🔬 Running Backtests...\n")

backtest_results = {}
for symbol in SYMBOLS:
    if symbol in market_data:
        print(f"Testing {symbol}...")
        
        try:
            # Initialize backtester
            backtester = Backtester(initial_balance=10000, leverage=5)
            
            # Run backtest
            results = backtester.run_backtest(
                symbol=symbol,
                timeframe=mt5.TIMEFRAME_H1,
                start_date=datetime(2023, 1, 1),
                end_date=datetime(2023, 12, 31),
                strategy_params={
                    'ema_period': 50,
                    'rsi_period': 14,
                    'macd_slow': 26,
                    'macd_fast': 12,
                    'macd_signal': 9
                }
            )
            
            backtest_results[symbol] = results
            
            if results:
                print(f"✅ {symbol} Backtest Results:")
                print(f"  Total Return: {results.get('total_return', 0):.2%}")
                print(f"  Win Rate: {results.get('win_rate', 0):.1f}%")
                print(f"  Total Trades: {results.get('total_trades', 0)}")
                print(f"  Profit Factor: {results.get('profit_factor', 0):.2f}")
                print(f"  Max Drawdown: {results.get('max_drawdown', 0):.2%}")
            else:
                print(f"❌ Backtest failed for {symbol}")
                
        except Exception as e:
            print(f"❌ Error backtesting {symbol}: {e}")
        print()

## 7. Performance Visualization

In [ ]:
# Visualize backtest results
if backtest_results:
    # Create performance summary
    performance_df = pd.DataFrame({
        'Symbol': list(backtest_results.keys()),
        'Total Return': [results.get('total_return', 0) for results in backtest_results.values()],
        'Win Rate': [results.get('win_rate', 0) for results in backtest_results.values()],
        'Total Trades': [results.get('total_trades', 0) for results in backtest_results.values()],
        'Profit Factor': [results.get('profit_factor', 0) for results in backtest_results.values()],
        'Max Drawdown': [results.get('max_drawdown', 0) for results in backtest_results.values()]
    })
    
    # Plot performance metrics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Total Return
    axes[0,0].bar(performance_df['Symbol'], performance_df['Total Return'] * 100)
    axes[0,0].set_title('Total Return (%)')
    axes[0,0].set_ylabel('Return %')
    
    # Win Rate
    axes[0,1].bar(performance_df['Symbol'], performance_df['Win Rate'])
    axes[0,1].set_title('Win Rate (%)')
    axes[0,1].set_ylabel('Win Rate %')
    
    # Total Trades
    axes[1,0].bar(performance_df['Symbol'], performance_df['Total Trades'])
    axes[1,0].set_title('Total Trades')
    axes[1,0].set_ylabel('Number of Trades')
    
    # Profit Factor
    axes[1,1].bar(performance_df['Symbol'], performance_df['Profit Factor'])
    axes[1,1].axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Break Even')
    axes[1,1].set_title('Profit Factor')
    axes[1,1].set_ylabel('Profit Factor')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Display performance table
    print("\n📊 Performance Summary:")
    print(performance_df.to_string(index=False, float_format='%.3f'))
else:
    print("❌ No backtest results to display")

## 8. Risk Analysis

In [ ]:
# Analyze risk metrics
print("⚠️ Risk Analysis:")
print("\nCurrent Risk Settings:")
print(f"• Leverage: 1:{os.getenv('LEVERAGE_RATIO', 5)}")
print(f"• Max Risk per Trade: {os.getenv('MAX_RISK_PERCENT', 2)}%")
print(f"• Max Daily Loss: {os.getenv('MAX_DAILY_LOSS', 5)}%")
print(f"• Max Open Positions: {os.getenv('MAX_OPEN_POSITIONS', 3)}")

# Calculate portfolio risk
if market_data:
    print("\n📊 Market Volatility Analysis:")
    volatilities = {}
    correlations = pd.DataFrame()
    
    for symbol in SYMBOLS:
        if symbol in market_data:
            df = market_data[symbol]
            returns = df['close'].pct_change().dropna()
            volatility = returns.std() * np.sqrt(24 * 365) * 100  # Annualized volatility
            volatilities[symbol] = volatility
            correlations[symbol] = returns
    
    # Display volatilities
    for symbol, vol in volatilities.items():
        print(f"• {symbol}: {vol:.2f}% annual volatility")
    
    # Show correlation matrix
    if len(correlations.columns) > 1:
        corr_matrix = correlations.corr()
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                   square=True, fmt='.3f')
        plt.title('Currency Pair Correlation Matrix')
        plt.show()
        
        print("\n🔗 Correlation Insights:")
        for i, symbol1 in enumerate(correlations.columns):
            for j, symbol2 in enumerate(correlations.columns):
                if i < j:
                    corr = corr_matrix.loc[symbol1, symbol2]
                    if abs(corr) > 0.7:
                        print(f"• {symbol1} vs {symbol2}: {corr:.3f} (High correlation - reduce simultaneous positions)")
                    elif abs(corr) < 0.3:
                        print(f"• {symbol1} vs {symbol2}: {corr:.3f} (Low correlation - good for diversification)")

## 9. Live Trading Monitor

In [ ]:
# Monitor current trading status
def get_current_positions():
    """Get current open positions"""
    try:
        positions = mt5.positions_get()
        if positions:
            positions_df = pd.DataFrame(list(positions), columns=positions[0]._asdict().keys())
            return positions_df
        return pd.DataFrame()
    except:
        return pd.DataFrame()

def get_current_orders():
    """Get current pending orders"""
    try:
        orders = mt5.orders_get()
        if orders:
            orders_df = pd.DataFrame(list(orders), columns=orders[0]._asdict().keys())
            return orders_df
        return pd.DataFrame()
    except:
        return pd.DataFrame()

# Current positions
positions = get_current_positions()
if not positions.empty:
    print("📈 Current Open Positions:")
    print(positions[['symbol', 'type', 'volume', 'price_open', 'price_current', 'profit']].to_string(index=False))
    
    total_profit = positions['profit'].sum()
    print(f"\nTotal Unrealized P&L: ${total_profit:.2f}")
else:
    print("📈 No open positions")

# Current orders
orders = get_current_orders()
if not orders.empty:
    print("\n📋 Pending Orders:")
    print(orders[['symbol', 'type', 'volume', 'price_open']].to_string(index=False))
else:
    print("\n📋 No pending orders")

# Recent trades (from log if available)
import os
if os.path.exists('trades.json'):
    try:
        import json
        with open('trades.json', 'r') as f:
            trades = json.load(f)
        if trades:
            print(f"\n📊 Recent Trades: {len(trades)} trades executed")
            # Show last 5 trades
            recent_trades = trades[-5:]
            for trade in recent_trades:
                print(f"  {trade.get('timestamp', '')} - {trade.get('symbol', '')} {trade.get('action', '')} - P&L: ${trade.get('profit', 0):.2f}")
    except:
        pass
else:
    print("\n📊 No trade history available")

## 10. Quick Actions

In [ ]:
# Quick actions for bot management
print("🎛️ Quick Actions:")
print("\nTo start the bot:")
print("  !python bot.py")
print("\nTo run backtest:")
print("  !python backtest.py")
print("\nTo test trading strategies:")
print("  !python test_trading.py")

print("\n📁 Important Files:")
print("  • trading_bot.log - Application logs")
print("  • trades.json - Trade history")
print("  • .env - Configuration settings")

# Check log file
if os.path.exists('trading_bot.log'):
    with open('trading_bot.log', 'r') as f:
        lines = f.readlines()
    print(f"\n📝 Last 3 log entries:")
    for line in lines[-3:]:
        print(f"  {line.strip()}")
else:
    print("\n📝 No log file found")

# Cleanup
if mt5.initialize():
    mt5.shutdown()

print("\n✅ Analysis complete! Your MT5 trading bot is ready for action.")